# Strategy analysis example

Debugging a strategy can be time-consuming. FreqTrade offers helper functions to visualize raw data.

## Setup

In [1]:
%matplotlib inline
import os
from pathlib import Path
# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/home/cryo/freqtrade"
i=0
try:
    os.chdirdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

# Customize these according to your needs.

# Define some constants
timeframe = "1m"
# Name of the strategy class
strategy_name = 'MLStrategy'
# Path to user data
user_data_dir = Path('user_data')
# Location of the strategy
strategy_location = user_data_dir / 'strategies'
# Location of the data
data_location = Path(user_data_dir, 'data', 'binance')
# Pair to analyze - Only use one pair here
pair = "BTC_USDT"

/home/cryo/freqtrade


In [2]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=timeframe,
                            pair=pair)

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
# candles.head()
candles.tail()

Loaded 43199 rows of data for BTC_USDT from user_data/data/binance


,date,open,high,low,close,volume
43194,2019-12-15 12:15:00+00:00,7127.91,7130.65,7126.59,7127.55,13.270077
43195,2019-12-15 12:16:00+00:00,7126.60,7130.00,7122.26,7128.39,28.326627
43196,2019-12-15 12:17:00+00:00,7128.76,7129.58,7123.33,7123.54,32.010987
43197,2019-12-15 12:18:00+00:00,7124.64,7125.26,7122.45,7124.30,9.992058
43198,2019-12-15 12:19:00+00:00,7124.45,7129.08,7124.30,7128.03,12.885772


## Load and run strategy
* Rerun each time the strategy file is changed

### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [17]:
# Load strategy using values set above
from freqtrade.resolvers import ExchangeResolver, StrategyResolver
from freqtrade.configuration import Configuration
from freqtrade.state import RunMode
from freqtrade.data.dataprovider import DataProvider
from freqtrade.strategy.interface import IStrategy

config = Configuration.from_files(["config.json"])
# print(config)
exchange = ExchangeResolver.load_exchange(config['exchange']['name'], config, False)
dataprovider = DataProvider(config, exchange)
IStrategy.dp = dataprovider
# dataframe = dataprovider.historic_ohlcv(pair)
strategy = StrategyResolver.load_strategy({'strategy': strategy_name,
                            'user_data_dir': user_data_dir,
                            'strategy_path': strategy_location})

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {'pair': pair})
print(df.head())

2020-01-08 14:45:15,955 - freqtrade.configuration.configuration - INFO - Using config: config.json ...
2020-01-08 14:45:15,957 - freqtrade.loggers - INFO - Verbosity set to 0
2020-01-08 14:45:15,958 - freqtrade.configuration.configuration - INFO - Using user-data directory: /home/cryo/freqtrade/user_data ...
2020-01-08 14:45:15,958 - freqtrade.configuration.configuration - INFO - Using data directory: /home/cryo/freqtrade/user_data/data/binance ...
2020-01-08 14:45:15,959 - freqtrade.configuration.check_exchange - INFO - Checking exchange...
2020-01-08 14:45:15,960 - freqtrade.configuration.check_exchange - INFO - Exchange "binance" is officially supported by the Freqtrade development team.
2020-01-08 14:45:15,961 - freqtrade.configuration.configuration - INFO - Using pairlist from configuration.
2020-01-08 14:45:15,962 - freqtrade.configuration.deprecated_settings - WARNING - DEPRECATED: Pairlists must be defined explicitly in the future.Defaulting to StaticPairList for now.
2020-01-0

init


2020-01-08 14:45:16,335 - freqtrade.data.converter - INFO - Missing data fillup for BTC/USDT: before: 43057 - after: 43200
2020-01-08 14:45:20,053 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using minimal_roi: {'80': 0.02, '40': 0.03, '20': 0.04, '0': 0.05}
2020-01-08 14:45:20,054 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using ticker_interval: 1m
2020-01-08 14:45:20,054 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using stoploss: -0.003
2020-01-08 14:45:20,055 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using trailing_stop: False
2020-01-08 14:45:20,055 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using trailing_stop_positive: 0.01
2020-01-08 14:45:20,056 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using trailing_stop_positive_offset: 0.02
2020-01-08 14:45:20,056 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using trailing_only_offset_is_reached: False
2020-01-08 14:45:20,057 - freqtrade.re

Model fitted using dataframe of length = 32311 during 4.069217205047607 seconds with testing score =  0.003153910205189292
predict:  [ 1.          1.          1.         -0.29765711] [-0.50041023]
                         date     open     high      low    close      volume  \
117 2019-11-15 14:18:00+00:00  8617.11  8619.19  8612.47  8615.19   59.393301   
118 2019-11-15 14:19:00+00:00  8615.00  8615.13  8590.00  8605.20  162.440927   
119 2019-11-15 14:20:00+00:00  8605.27  8612.59  8597.51  8609.33   71.135806   
120 2019-11-15 14:21:00+00:00  8605.97  8609.28  8582.41  8598.99  101.863293   
121 2019-11-15 14:22:00+00:00  8598.99  8613.09  8596.09  8602.39   74.981213   

          macd  macdsignal   minus_di        rsi  ...          sar  \
117  -8.693440   -8.700799  31.147201  37.663990  ...  8637.294200   
118  -9.588360   -8.878311  39.674759  33.246370  ...  8636.690716   
119  -9.850779   -9.072805  36.242630  36.559206  ...  8634.823087   
120 -10.768963   -9.412036  39.07300

In [18]:
from plotly.subplots import make_subplots
from plotly.offline import plot
import plotly.graph_objects as go

data = df.set_index('date', drop=False)
# data = df

data_red = data['2019-11-17':'2019-11-17']
fig = go.Figure()
fig2 = go.Figure()
# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red.close, name='sentiment'))
# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red['macd'], name='macd'))
# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red.minus_di, name='minus_di'))
# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red['rsi'], name='rsi'))
# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red.fastd, name='fastd'))
# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red.fastk, name='fastk'))
# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red.sar, name='sar'))

fig2.add_trace(go.Scatter(x=data_red.date, y=data_red['tema_long'], name='tema_long'))
fig2.add_trace(go.Scatter(x=data_red.date, y=data_red['tema_short'], name='tema_short'))
fig2.add_trace(go.Scatter(x=data_red.date, y=data_red['tema_base'], name='tema_base'))
fig2.add_trace(go.Scatter(x=data_red.date, y=data_red.close, name='close'))

# fig2.add_trace(go.Scatter(x=data_red.date, y=data_red.TRANGE, name='TRANGE'))
fig.add_trace(go.Scatter(x=data_red.date, y=data_red.perc_change, name='change, %'))
fig.add_trace(go.Scatter(x=data_red.date, y=data_red.future_perc_change, name='future change, %'))

# print(data_red.keys())

# fig2.show()
fig.show()


## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [5]:
from freqtrade.data.btanalysis import load_backtest_data

# Load backtest results
trades = load_backtest_data(user_data_dir / "backtest_results/backtest-result.json")

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

pair      sell_reason       
BTC/USDT  sell_signal           40
          roi                   20
          force_sell             1
          trailing_stop_loss     1
Name: sell_reason, dtype: int64

### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [6]:
# from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
# trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
# trades.groupby("pair")["sell_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [7]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
# parallel_trades = analyze_trade_parallelism(trades, '5m')


# parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [8]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

data_red = data['2019-11-11':'2019-11-14']
# data_red = data
data_red = data_red.reset_index(drop=True)
print(data_red.head())
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades,
                                   indicators1=['close'],
                                   indicators2=['sentiment', 'perc_change', 'future_perc_change']
                                  )




2020-01-08 14:37:32,569 - freqtrade.plot.plotting - WARNING - No buy-signals found.
2020-01-08 14:37:32,571 - freqtrade.plot.plotting - WARNING - No sell-signals found.


Empty DataFrame
Columns: [date, open, high, low, close, volume, macd, macdsignal, minus_di, rsi, fisher_rsi, fisher_rsi_norma, fastd, fastk, sar, sma, tema_base, tema_long, tema_short, TRANGE, perc_change, future_perc_change, buy, sell]
Index: []

[0 rows x 24 columns]


2020-01-08 14:37:32,713 - freqtrade.plot.plotting - INFO - Indicator "sentiment" ignored. Reason: This indicator is not found in your strategy.


In [9]:
# Show graph inline
# graph.show()

# Render graph in a seperate window
graph.show(renderer="browser")


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.